# Generate New Split of the Dataset
This code could be used to generate new random split of the dataset for training, validation and testing. The split would base on a ratio of 8:1:1 for training, validation and testing and would guarantee each slot would appear at least once in validation and testing set. 

In [ ]:
# load all the data
import json
train = json.load(open('data/covid/raw/train.json'))
dev = json.load(open('data/covid/raw/dev.json'))
test = json.load(open('data/covid/raw/test.json'))

In [ ]:
# load ontology
import dataset
from utils import load_dataset
dataset, ontology, vocab, Eword = load_dataset()

In [ ]:
# combine dataset
total = train+dev+test

In [ ]:
# figure out where each slot appear in the dialogs
slot_diag = dict(zip(ontology.slots, [[]]*len(ontology.slots)))
for diag in total:
    for turn in diag['turns']:
        for p, v in turn['turn_label']:
            if diag['dialogue_index'] not in slot_diag[p]:
                temp = []
                temp = temp + slot_diag.get(p)
                temp.append(diag['dialogue_index'])
                slot_diag[p] = temp

In [ ]:
# sort the tatal dataset by dialogue_index
total.sort(key=lambda x:x['dialogue_index'])

In [ ]:
# note that there's no dialog with index 462
total[461]

In [ ]:
# for each slot, put a dialog in dev and one in test
test = []
dev = []
ocp = []  #stands for the dialog already been used in dev/test
for slot, value in slot_diag.items():
    if len(value) >= 3:
        first = 0
        print()
        flag = True
        while flag == True:
            if value[first] <= 461:
                if total[value[first]-1] not in ocp:
                    test.append(total[value[first]-1])
                    ocp.append(total[value[first]-1])
                    flag = False
                elif total[value[first]-1] not in test:
                    first = first + 1
                else:
                    flag = False
            else:
                if total[value[first]-2] not in ocp:
                    test.append(total[value[first]-2])
                    ocp.append(total[value[first]-2])
                elif total[value[first]-2] not in test:
                    first = first + 1
                else: 
                    flag = False
        
        sec = 1
        flag = True
        while flag == True:
            if value[sec] <= 461:
                if total[value[sec]-1] not in ocp:
                    dev.append(total[value[sec]-1])
                    ocp.append(total[value[sec]-1])
                    flag = False
                elif total[value[sec]-1] not in dev:
                    sec = sec + 1
                else:
                    flag = False
            else:
                if total[value[sec]-2] not in ocp:
                    dev.append(total[value[sec]-2])
                    ocp.append(total[value[sec]-2])
                    flag = False
                elif total[value[sec]-2] not in dev:
                    sec = sec + 1
                else:
                    flag = False

In [ ]:
# remove the taken dialog from tatal
for diag in test:
    total.remove(diag)
    
for diag in dev:
    total.remove(diag)

In [ ]:
import sklearn.model_selection
trn_dev, test_set = sklearn.model_selection.train_test_split(total, train_size=len(total)-(60-len(test)), test_size=60-len(test))
train_set, dev_set = sklearn.model_selection.train_test_split(trn_dev, train_size=482, test_size=len(trn_dev)-482)

In [ ]:
# append the test set with previous slots
test_set += test

In [ ]:
# append the dev set with previous slots
dev_set += dev

In [ ]:
# save the split
json.dump(train_set, open(f'train.json', 'w'), indent=4)
json.dump(dev_set, open(f'dev.json', 'w'), indent=4)
json.dump(test_set, open(f'test.json', 'w'), indent=4)

In [ ]:
# update ontology
ont = {"slots": [], "system_acts": ["request_infomation","dontworry","need_lab_test","contact_doctor","concrete_advice"], "values": {}}
for slot in ontology.slots:
    if len(slot_diag[slot]) >= 3:
        ont["slots"].append(slot)
        ont["values"].update({slot: ontology.values[slot]})

In [ ]:
json.dump(ont, open(f'ontology.json', 'w'), indent=4)